<a href="https://colab.research.google.com/github/Ajithkumar7373/Movie-Recomdation/blob/main/Recomandation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Required libraries**

In [5]:
import numpy as np
import pandas as pd
import difflib # used to get a close match
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

**Load and Preprocessing**

In [6]:
#Load the csv file into dataframe
movies_data = pd.read_csv('movies.csv')

In [7]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [8]:
movies_data.shape

(4803, 24)

In [9]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [10]:
movies_data.isnull().sum()

,0
index,0
budget,0
genres,28
homepage,3091
id,0
keywords,412
original_language,0
original_title,0
overview,3
popularity,0


In [11]:
# Select relavent feature for recomandation
selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [12]:
# replace the null values with null string for relevent  features
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [13]:
# creat a new dataframe for slected features


In [14]:
# Combain the relavent featues
combained_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [15]:
combained_features

,0
0,Action Adventure Fantasy Science Fiction cultu...
1,Adventure Fantasy Action ocean drug abuse exot...
2,Action Adventure Crime spy based on novel secr...
3,Action Crime Drama Thriller dc comics crime fi...
4,Action Adventure Science Fiction based on nove...
...,...
4798,Action Crime Thriller united states\u2013mexic...
4799,Comedy Romance A newlywed couple's honeymoon ...
4800,Comedy Drama Romance TV Movie date love at fir...
4801,A New Yorker in Shanghai Daniel Henney Eliza...


In [16]:
# convert text  data to numeric data using text vectorizer
vectorizer = TfidfVectorizer()
feature_vectors=vectorizer.fit_transform(combained_features)

In [17]:
print(feature_vectors)

  (0, 201)	0.07860022416510505
  (0, 274)	0.09021200873707368
  (0, 5274)	0.11108562744414445
  (0, 13599)	0.1036413987316636
  (0, 5437)	0.1036413987316636
  (0, 3678)	0.21392179219912877
  (0, 3065)	0.22208377802661425
  (0, 5836)	0.1646750903586285
  (0, 14378)	0.33962752210959823
  (0, 16587)	0.12549432354918996
  (0, 3225)	0.24960162956997736
  (0, 14271)	0.21392179219912877
  (0, 4945)	0.24025852494110758
  (0, 15261)	0.07095833561276566
  (0, 16998)	0.1282126322850579
  (0, 11192)	0.09049319826481456
  (0, 11503)	0.27211310056983656
  (0, 13349)	0.15021264094167086
  (0, 17007)	0.23643326319898797
  (0, 17290)	0.20197912553916567
  (0, 13319)	0.2177470539412484
  (0, 14064)	0.20596090415084142
  (0, 16668)	0.19843263965100372
  (0, 14608)	0.15150672398763912
  (0, 8756)	0.22709015857011816
  :	:
  (4801, 403)	0.17727585190343229
  (4801, 4835)	0.24713765026964
  (4801, 17266)	0.28860981849329476
  (4801, 13835)	0.27870029291200094
  (4801, 13175)	0.28860981849329476
  (4801, 171

**Cosine Similarity**

In [18]:
# Find the similarity score using cosine similarity
similarity = cosine_similarity(feature_vectors)

In [19]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [20]:
# getting the movie name from user
movie_name = input('Enter your favourite movie name: ')

Enter your favourite movie name: avatar


In [21]:
# create a list for all movies
list_of_all_movies = movies_data['title'].tolist()


In [22]:
list_of_all_movies

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [23]:
# Find the close math  for the movie name given by the user
find_close_math = difflib.get_close_matches(movie_name,list_of_all_movies)

In [24]:
find_close_math

['Avatar']

In [25]:
close_math=find_close_math[0]


In [26]:
# find index of the movie
index_of_movie = movies_data[movies_data.title == close_math]['index'].values[0]

In [27]:
print(index_of_movie)

0


In [28]:
# getting a list of similar movies
similarity_score = list(enumerate(similarity[index_of_movie]))#enumerate for loop the  list to check the index value


In [29]:
similarity_score

[(0, 1.0),
 (1, 0.07219486822992488),
 (2, 0.037732999577179294),
 (3, 0.012520204623868908),
 (4, 0.10702574467235304),
 (5, 0.07786899789424222),
 (6, 0.008237143013608844),
 (7, 0.03613473061484885),
 (8, 0.02960930964063025),
 (9, 0.026287167439951735),
 (10, 0.09261074046755373),
 (11, 0.012717759249124133),
 (12, 0.02721736008310012),
 (13, 0.029569752523347516),
 (14, 0.06915925473724743),
 (15, 0.019551594499309027),
 (16, 0.034263405780616416),
 (17, 0.02603656461429414),
 (18, 0.05714759266672412),
 (19, 0.0389505953521203),
 (20, 0.03971480215415496),
 (21, 0.01201480380565613),
 (22, 0.030438694261989592),
 (23, 0.045926535588179496),
 (24, 0.04623989017965258),
 (25, 0.04284926095950226),
 (26, 0.07010711150614288),
 (27, 0.037198076232328854),
 (28, 0.04083909796927843),
 (29, 0.03858648330156398),
 (30, 0.07893753610792024),
 (31, 0.06055522138055148),
 (32, 0.030362745635800832),
 (33, 0.03591021470068869),
 (34, 0.0),
 (35, 0.037696741034748446),
 (36, 0.04891087950911

In [30]:
len(similarity_score)

4803

In [31]:
# Sorting the movies base on similarity
sorted_similar_movies = sorted(similarity_score,key=lambda x:x[1],reverse=True)


In [32]:
print(sorted_similar_movies)

[(0, 1.0), (3158, 0.2494676630753241), (2403, 0.24841462595906272), (94, 0.24505931974059822), (56, 0.20378069648285427), (47, 0.20115287461144912), (1053, 0.19702752258651424), (838, 0.18017023369312363), (3730, 0.1764624118531341), (4593, 0.17448845797415172), (239, 0.17441748680810656), (1531, 0.1682605817219649), (2696, 0.1650346025917652), (812, 0.16062301907491794), (643, 0.15644455512484967), (4401, 0.15468923545220403), (2198, 0.15217161971893786), (770, 0.1502572672775349), (1951, 0.14933372705282924), (2229, 0.14661801285492254), (1922, 0.14481974301913123), (206, 0.14226144606175548), (3208, 0.1401230206493547), (1759, 0.13899056016968867), (43, 0.1348209130228474), (1473, 0.13476547670086914), (278, 0.13291021545504), (158, 0.1325289213162767), (1650, 0.13024318650645417), (1275, 0.1260221630479115), (3439, 0.12480340331169379), (661, 0.12153002734138184), (3202, 0.12144749322246055), (4332, 0.12002556168548509), (3105, 0.11948466494212537), (775, 0.11847062758014923), (109

In [33]:
# prin the name of similar movies based on index
print("Movies suggested for you: \n")
i=1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if(i<11):
    print(i,'.',title_from_index)
    i+=1

Movies suggested for you: 

1 . Avatar
2 . Alien
3 . Aliens
4 . Guardians of the Galaxy
5 . Star Trek Beyond
6 . Star Trek Into Darkness
7 . Galaxy Quest
8 . Alien³
9 . Cargo
10 . Trekkies


**Recomandation System**

In [34]:
def recommandation_system(movie_name):
# create a list for all movies
  list_of_all_movies = movies_data['title'].tolist()

# Find the close math  for the movie name given by the user
  find_close_math = difflib.get_close_matches(movie_name,list_of_all_movies)

  close_math=find_close_math[0]

# find index of the movie
  index_of_movie = movies_data[movies_data.title == close_math]['index'].values[0]

# getting a list of similar movies
  similarity_score = list(enumerate(similarity[index_of_movie]))#enumerate for loop the  list to check the index value

# Sorting the movies base on similarity
  sorted_similar_movies = sorted(similarity_score,key=lambda x:x[1],reverse=True)

# prin the name of similar movies based on index
  print("Movies suggested for you: \n")
  i=1
  for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies_data[movies_data.index==index]['title'].values[0]
    if(i<11):
      print(i,'.',title_from_index)
      i+=1

In [35]:
print(recommandation_system('Avatar'))

Movies suggested for you: 

1 . Avatar
2 . Alien
3 . Aliens
4 . Guardians of the Galaxy
5 . Star Trek Beyond
6 . Star Trek Into Darkness
7 . Galaxy Quest
8 . Alien³
9 . Cargo
10 . Trekkies
None


In [36]:
import pickle
with open('movies.pkl','wb') as f:
  pickle.dump(similarity,f)